<a href="https://colab.research.google.com/github/arkincognito/ML-from-scrap/blob/master/Computational_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is based on an assignment from DS School's Deep Learning Course.

Computational Graph enables Partial Derivation without explicitly representing the partial derivative.

In this notebook, I'll write computational graph code for

*Nodes(Log, Square, Trigonometric Functions)
and
*Loss Functions(Mean Square Error, Cross Entropy)

## Computational Graph 복습

computational graph는 계산과정을 그래프로 나타난 것으로, 노드는 연산을 엣지는 입출력 관계를 의미합니다. computational graph를 통해 아무리 복잡한 계산도 각 노드별로의 local gradient만 계산하기 때문에 효율적으로 계산이 가능합니다. 각 노드는 자신과 관련된 계산 외에는 고려하지 않고 중간 미분 결과를 공유하여 다수의 미분을 효율적으로 계산합니다. chain rule의 과정을 시각적으로 표현한 것이라고도 볼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=14x4zQpEEatgMb1W0BY47lXKjM5haZq1x" width="600">


>- **Forward Propagation**

>  **순전파(forward propagation)**는 뉴럴 네트워크의 입력층부터 출력층까지 순서대로 변수들을 계산하고 저장합니다. 그래프 상에서 왼쪽에서 오른쪽으로 진행되는 파란색으로 표시된 계산입니다.
  

>- **Back Propagation**

>  **역전파(back propagaton)**는 네트워크 전체에 대해 반복적으로 chain rule을 적용하여 gradient를 계산하여 뉴럴 네트워크를 효율적으로 학습하는 데에 사용되는 알고리즘입니다. 오차를 역 방향으로 전파하는 방법이라 오차역전파법이라고도 말합니다. 중간 변수와 가중치에 대한 gradient를 계산하고 저장합니다. 그래프 상에서 오른쪽에서 왼쪽으로 진행되는 빨간색으로 표시된 계산입니다.

#### Multiply 노드의 Forward/Back Propagation를 구하는 Class를 아래와 같이 만들어 보았습니다. 

- #### Multiply 노드

곱셈 노드의 수식이 $z=f(x,y)=x\times y $일 때 $z$의 gradient는 아래와 같습니다. 

$$
\nabla z = \left ( \frac{\partial z}{\partial x},\frac{\partial z}{\partial y}  \right ) = \left ( \frac{\partial (xy)}{\partial x},\frac{\partial (xy)}{\partial y}  \right ) = (y,x)
$$

<img src="http://drive.google.com/uc?export=view&id=1pQ9HFmr9_31daD8YIhO72IQtr5Kvj2GP" width="600">


>- **Forward Propagation**

>  곱셈 노드는 입력된 두 가지 값을 곱하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  곱셈 노드는 입력된 값에 forward propagation 당시의 입력 변수들을 서로 바꾸어 곱하여 다음 노드로 전달합니다.

In [ ]:
# 곱셈 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Multiply:
    # f(x,y) = xy
    def forward(self, x, y):
        self.x = x
        self.y = y
        # 두 가지 값을 곱하여 다음 노드로 전달합니다.
        return self.x * self.y
    # dfdx = y, dfdy = x
    def backward(self):
        dx = self.y
        dy = self.x
        # forward propagation의 입력 변수들을
        # 서로 바꾸어 곱하여 다음 노드로 전달합니다.
        return dx, dy

In [ ]:
multiply=Multiply()
forward2 = multiply.forward(10,3)
forward2

30

In [ ]:
multiply.backward()

(3, 10)

More generally, multiplication can be written as $z = f(x_0, x_1, ... , x_{n-1}) = $$\prod_{i=0}^{n-1} x_i$.

Then the gradient of $z$ is:

$$ \nabla z = \left ( \frac{\partial z}{\partial x_0},\frac{\partial z}{\partial x_1}, ... \frac{\partial z}{\partial x_{n-1}} \right ) = \left ( \frac{\partial (\prod_{i=0}^{n-1} x_i)}{\partial x_0},\frac{\partial (\prod_{i=0}^{n-1} x_i)}{\partial x_1}, ... ,\frac{\partial (\prod_{i=0}^{n-1} x_i)}{\partial x_{n-1}}  \right )$$

Thus,

$$ \frac{\partial z}{\partial x_j} = \frac{\prod_{i=0}^{n-1} x_i} {x_j} $$

Note that the input x now is the list of numbers we're solving for product.

In [ ]:
class MultiplyAll:
    def forward(self, array):
        self.x = np.array(array)
        return np.product(self.x)
    def backward(self):
        return np.array(list(np.prod(self.x) / i for i in self.x))

### 1. 로그함수 노드 구현해보기

자연대수 $e$를 밑으로 하는 로그함수 노드의 수식이 $z=f(x)=log(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial (log(x))}{\partial x}\right ) = \frac{1}{x}
$$

<img src="http://drive.google.com/uc?export=view&id=1YKou4QvGFwjWV_zzk0H8cMS0vGT7lFMB" width="600">

>- **Forward Propagation**

>  로그 노드는 입력된 값을 밑이 $e$인 로그함수의 진수로 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  로그 노드는 입력된 값에 역수를 취한 값을 곱하여 다음 노드로 전달합니다.

In [ ]:
# 로그를 계산할 수 있는 numpy 패키지를 가져옵니다. np라는 축약어로 사용합니다. 
import numpy as np

In [ ]:
# 로그 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Log:
    
    # f(x) = log(x)
    def forward(self, x):
        self.x = x
        return np.log(self.x)
        
    # dfdx = 1/x
    def backward(self):
        return 1.0/self.x

log 노드 객체를 생성하고, **x=2**값을 넣어준 후 forward, backward값이 각각 **0.693, 0.5** 으로 잘 구해지는지 확인해 봅시다.

In [ ]:
log = Log()
x = 2
log.forward(x), log.backward()

(0.6931471805599453, 0.5)

### 2. 제곱(Square) 노드 구현해보기

제곱 노드의 수식이 $z=f(x)=x^2$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial x^2}{\partial x}\right ) = 2x
$$

<img src="http://drive.google.com/uc?export=view&id=1C67JqOdnW4dxbLBVHLxF8Hvrr2NoTth3" width="600">


>- **Forward Propagation**

>  로그 노드는 입력된 값에 제곱을 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  로그 노드는 입력된 값에 $2x$를 곱하여 다음 노드로 전달합니다.

In [ ]:
# 제곱 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Square:
    
    # f(x) = x^2
    def forward(self, x):
        self.x = x
        return self.x ** 2
    # dfdx = 2x
    def backward(self):
        return 2 * self.x

square 노드 객체를 생성하고, **x=5**값을 넣어준 후 forward, backward값이 각각 **25, 10** 으로 잘 구해지는지 확인해 봅시다.

In [ ]:
square = Square()
x =5
square.forward(x), square.backward()

(25, 10)

### 3. 삼각함수(sin, cos, tan) 노드 구현해보기

#### 3-1. sin함수 노드 구현해보기
sin함수 노드의 수식이 $z=f(x)=sin(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial sin(x)}{\partial x}\right ) = cos(x)
$$
<img src="http://drive.google.com/uc?export=view&id=1zdwScJP5hbMTtJETFVo1P9F8ZACoU-fc" width="600">

>- **Forward Propagation**

>  sin 노드는 입력된 값에 sin함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  sin 노드는 입력된 값에 $cos(x)$를 곱하여 다음 노드로 전달합니다.

In [ ]:
# sin 함수 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class Sin:
    
    def forward(self, x):
        self.x = x
        return np.sin(x)

    def backward(self):
        return np.cos(x)

sin 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **0.866, 0.5** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [ ]:
sin = Sin()
x = np.pi / 3
sin.forward(x), sin.backward()

(0.8660254037844386, 0.5000000000000001)

#### 3-2. cos함수 노드 구현해보기
cos함수 노드의 수식이 $z=f(x)=cos(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial cos(x)}{\partial x}\right ) = -sin(x)
$$

<img src="http://drive.google.com/uc?export=view&id=11AEEdSWOmBjGQhzW2-BDboXFjCvd-hi8" width="600">


>- **Forward Propagation**

>  cos 노드는 입력된 값에 cos함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  cos 노드는 입력된 값에 $-sin(x)$를 곱하여 다음 노드로 전달합니다.

In [ ]:
class Cos:
    
    def forward(self, x):
        self.x = x
        return np.cos(x)
    
    def backward(self):
        return -np.sin(x)

cos 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **0.5, -0.866** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [ ]:
cos = Cos()
x = np.pi/3
cos.forward(x), cos.backward()

(0.5000000000000001, -0.8660254037844386)

#### 3-3. tan함수 노드 구현해보기
tan함수 노드의 수식이 $z=f(x)=tan(x)$일 때 $z$의 gradient는 아래와 같습니다.


$$
\nabla z = \left ( \frac{\partial z}{\partial x}\right ) = \left ( \frac{\partial tan(x)}{\partial x}\right ) = \frac{1}{ cos(x)^2}
$$

<img src="http://drive.google.com/uc?export=view&id=16Q1LDQ4L2uY8dkqKgMRZP8UILcKtG10L" width="600">


>- **Forward Propagation**

>  tan 노드는 입력된 값에 tan함수를 취하여 다음 노드로 전달합니다.
  

>- **Back Propagation**

>  tan 노드는 입력된 값에 $\frac{1}{ cos(x)^2}$를 곱하여 다음 노드로 전달합니다.

In [ ]:
class Tan:
    
    def forward(self, x):
        self.x = x
        return np.tan(x)
    
    def backward(self):
        return 1 / (np.cos(x)**2)

cos 노드 객체를 생성하고, **x=60&deg;**값을 넣어준 후 forward, backward값이 각각 **1.732, 4** 으로 잘 구해지는지 확인해 봅시다.
(numpy의 pi를 사용하여 x=np.pi / 3 로 넣어 줄 수 있습니다.)

In [ ]:
tan = Tan()
x = np.pi/3
tan.forward(x), tan.backward()

(1.7320508075688767, 3.9999999999999982)

#### 이제, 그동안 다뤄봤었던 Loss Function을 Computational Graph로 그리고 이를 이용하여 Loss Function의 편미분을 구현해 보겠습니다. 

### 4. MSE Loss Function 노드 구현해보기

Regression 모델에서 주로 사용하는 평균제곱오차(MSE; Mean Squared Error) Loss Function (데이터 한 개의 Error)은 다음과 같이 나타낼 수 있습니다. (여기서는  $\frac{1}{2}$을 추가해주었는데 경우에 따라 생략하기도 합니다.)

$$
MSE=\frac{1}{2} (\hat{y}-y)^2
$$


데이터의 총 갯수를 n개라 하고, $y^{(i)}$는 i번째 데이터의 label값, $\hat{y}^{(i)}$은 ${y}^{(i)}$의 예측치로 $\hat{y}^{(i)}=\sigma(w^Tx^{(i)}+b)$라 하고 MSE의 Cost Function(비용함수:모든 Loss의 평균)를 나타내면 다음과 같습니다. 


$$
MSE(Cost)=\frac{1}{n}\sum_{i=1}^{n}\frac{1}{2} (\hat{y}^{(i)}-y^{(i)})^2
$$

Cost Function은 Loss Function의 단순 평균이므로, 여기서는 Loss Function만 구현해 보도록 하겠습니다.

위의 MSE Loss Function을 Computatioanl Graph로 그리면 다음과 같습니다. 

<img src="http://drive.google.com/uc?export=view&id=1dqfw6Tpeo6CkNsns8IMU3Bw1ItkAUNZ6" width="600">


이 Loss Function을 Gradient Descent 알고리즘에 적용하여 최적의 w와 b를 구해주기 위해서는 Loss 값을 w와 b로 편미분한 값을 필요로 했습니다. 따라서, 우리는 위 MSE Computational Graph에서 예측치인 $\hat{y}$에 대한 Loss의 편미분 값. $\frac{\partial L}{\partial \hat{y}} $ 을 구해주어야 합니다. 
즉, 지금 우리가 만들어주려하는 MSE 노드는 다음과 같이 나타낼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=16dbnhIahsvpVh1eAF4uavMA8mZIMkWhF" width="600">


이번 4번 과제는 위와 같이 MSE 노드에 $\hat{y}$과 $y$를 입력받았을 때, forward propagation으로 $ MSE=\frac{1}{2} (\hat{y}-y)^2 $ 을 출력하고, backward propagation으로는 $\frac{\partial L}{\partial \hat{y}} $ 값을 return 해주는 MSE 노드를 만드는 것입니다.

In [ ]:
class Add:
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x + y
    def backward(self):
        return 1.0, 1.0

In [ ]:
class MSE:
    mult1 = Multiply()
    mult2 = Multiply()
    add = Add()
    sq = Square()
    def forward(self, y_predict, y):
        self.y_predict = y_predict
        self.y = y
        fw1 = self.mult1.forward(self.y, -1)
        fw2 = self.add.forward(fw1, self.y_predict)
        fw3 = self.sq.forward(fw2)
        fw4 = self.mult2.forward(fw3, 1/2)
        return fw4
    
    def backward(self):
        bw1 = self.mult2.backward()[0]
        bw2 = self.sq.backward() * bw1
        bw3 = self.add.backward()[1] * bw2
        return bw3

MSE 노드 객체를 생성하고, **y_predict = 1, y = 4**값을 넣어준 후 forward, backward값이 각각 **4.5, -3.0** 으로 잘 구해지는지 확인해 봅시다.

In [ ]:
mse = MSE()
mse.forward(1,4), mse.backward()

(4.5, -3.0)

### 5. Cross Entropy Loss Function 노드 구현해보기

Cross Entropy Loss Function은 다음과 같이 나타낼 수 있습니다.

class(label)의 총 갯수를 C개라 하고, $y_{c}$는 c 번째 label 값, $\hat{y}_{c}$은 ${y}_{c}$의 예측치로 $\hat{y}_{c}=\sigma(w^Tx_{c}+b)$라 하고 Cross Entropy Loss를 나타내면 다음과 같습니다. 

$$
\text{Cross Entropy Loss} = -\sum_{c=1}^{C}  y_{c} \times log(\hat{y}_{c})
$$

Binary Classification에서는 C값이 2이므로 다음과 같이 나타낼 수 있습니다.

(C값이 1이라고 생각하실 수 있지만, Cross Entropy Loss를 적용해줄 때는 원핫인코딩된 y를 사용하므로 Yes / No 두 개의 label이 생성됩니다. 그리고, $y_{1}+y_{2}=1$ 이므로 $y_{1}$을 그냥 $y$로 나타내면 아래와 같이 익숙한 Loss Function이 나옵니다. )

$$ \text{Cross Entropy Loss(binary)} = - y \times log(\hat{y}) - (1-{y}) \times log(1-\hat{y})$$

이번 과제에서는 Binary Classification이 아닌 Multi-Class, 즉, C=3인 Multi-Class Classification 의 Cross Entropy Loss를 Computational Graph로 그리고, 이 Cross Entropy Loss의 편미분을 구현해보도록 하겠습니다. 

C=3를 대입한 Cross Entropy Loss는 다음과 같고, 

$$
\text{Cross Entropy Loss} = -\sum_{c=1}^{3}  y_{c} \times log(\hat{y}_{c})
$$

이를 Computational Graph로 그려주면 다음과 같습니다. 

<img src="http://drive.google.com/uc?export=view&id=1HFLacOYnnN5ihTN9pm1_LEL1CB-1JNny" width="600">


이 Loss Function을 Gradient Descent 알고리즘에 적용하여 최적의 w와 b를 구해주기 위해서는 Loss 값을 w와 b로 편미분한 값을 필요로 했습니다. 따라서, 우리는 위 Cross Entropy의 Computational Graph에서 예측치인 $\hat{y}$에 대한 Loss의 편미분 값. $\frac{\partial L}{\partial \hat{y}} $ 을 구해주어야 합니다. 
즉, 지금 우리가 만들어주려하는 CE 노드는 다음과 같이 나타낼 수 있습니다. 

<img src="http://drive.google.com/uc?export=view&id=15wLtFHEr884R75PZrl7TLTUj2izJkWM-" width="600">


이번 5번 과제는 위와 같이 CE(Cross Entropy) 노드에 $\hat{y}$과 $y$를 입력받았을 때, forward propagation으로 $ \text{Cross Entropy Loss} = -\sum_{c=1}^{3}  y_{c} \times log(\hat{y}_{c})$ 을 출력하고, backward propagation으로는 $\frac{\partial L}{\partial \hat{y}} $ 값을 return 해주는 CE 노드를 만드는 것입니다.

In [ ]:
mult = Multiply()
a = np.array([2,3,4])
b = np.array([0,1,2])
mult.forward(a,b), mult.backward()

(array([0, 3, 8]), (array([0, 1, 2]), array([2, 3, 4])))

In [ ]:
log = Log()
a = np.array([1,2,10])
log.forward(a), log.backward()

(array([0.        , 0.69314718, 2.30258509]), array([1. , 0.5, 0.1]))

In [ ]:
class AddAll():
    def forward(self, array):
        self.array = np.array(array)
        return self.array.sum()
    
    def backward(self):
        return np.array([1 for i in range (self.array.shape[0])])

In [ ]:
# CE 노드에 적용되는
# forward, back propagation 메소드를 정의합니다.
class CE:
    log = Log()
    mult = Multiply()
    addAll = AddAll()
    lastMult = Multiply()
    def forward(self, y_predict_ce, y_ce):
        self.y_predict_ce = np.array(y_predict_ce)
        self.y_ce = np.array(y_ce)
        log_y_predict = self.log.forward(self.y_predict_ce)
        log_yp_times_y = self.mult.forward(log_y_predict, self.y_ce)
        added = self.addAll.forward(log_yp_times_y)
        multiplied = self.lastMult.forward(added, -1)
        return multiplied
    
    def backward(self):
        bw1 = self.lastMult.backward()[0]
        bw2 = bw1 * self.addAll.backward()
        bw3 = bw2 * self.mult.backward()[0]
        bw4 = bw3 * self.log.backward()
        return bw4

CE 노드 객체를 생성하고, **y_predict_ce = [2, 3, 4] , y_ce = [0, 1, 2]** 값을 넣어준 후 forward, backward값이 각각 **-3.8712, (0, -0.33, -0.5)** 으로 잘 구해지는지 확인해 봅시다.

In [ ]:
ce = CE()
y_predict_ce = [2, 3, 4]
y_ce = [0, 1, 2]
print(f'Cross Entropy: {ce.forward(y_predict_ce, y_ce)}\nBackward Propagation: {ce.backward()}')

Cross Entropy: -3.8712010109078907
Backward Propagation: [ 0.         -0.33333333 -0.5       ]
